# Calculation of sensitivity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as spio
import tensorflow as tf
import tensorflow_addons as tfa
from os.path import join as osj
import pandas as pd


import random
import pickle
import time
import os
import argparse
import copy

from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Differential privacy libraries
from diffprivlib import mechanisms
from diffprivlib import models
from diffprivlib import tools
from diffprivlib.accountant import BudgetAccountant
from diffprivlib.utils import check_random_state
from diffprivlib.mechanisms import Laplace, LaplaceBoundedNoise, GaussianAnalytic
from diffprivlib.mechanisms import DPMechanism

from collections import Counter

import logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger()


In [ ]:
beats = [] 
filename = 
dict_samples = spio.loadmat('../data/s2s_mitbih_aami.mat')

samples = dict_samples['s2s_mitbih'] # 2D array with 2 columns: ecg values and labels
values = samples[0]['seg_values'] # ecg values
labels = samples[0]['seg_labels'] # labels

In [ ]:
def get_global_sensitivity(dict_patients, values, labels):
    
    with open(osj("..", "data", "all_patients.pkl"), "rb") as f:
        dict_patients = pickle.load(f)
    
    # Step 1: Get counts per patient
    all_count = {patient: {"F": 0, "N": 0, "S": 0, "V": 0, "Q": 0, "min": 0.0, "max": 0.0, "mean": 0.0} for patient in dict_patients}
    counter = 0

    for patient in dict_patients:

        label_counts = Counter()
        patient_values = []

        # count labels per patient
        label_string = labels[counter][0]
        label_counts = dict(Counter(label_string))
        for key in ["F", "N", "S", "V", "Q"]:
            try:
                all_count[patient][key] = label_counts[key] 
            except KeyError:
                all_count[patient][key] = 0

        # count values per patient
        for segment in values[counter]: 
            for segment_values in segment[0]: 
                patient_values.extend(segment_values) 

        # get min, max and mean values per patient
        if patient_values:
            all_count[patient]["min"] = np.min(patient_values)
            all_count[patient]["max"] = np.max(patient_values)
            all_count[patient]["mean"] = np.mean(patient_values)
        counter += 1

    # Step 2: aggregate values for all patients
    all_patients_aggregated = {"F": 0, "N": 0, "S": 0, "V": 0, "Q": 0, "g_min": 0.0, "g_max": 0.0, "g_mean": 0.0}

    for patient in all_count:
        for key in ["F", "N", "S", "V", "Q"]:
            all_patients_aggregated[key] += all_count[patient][key]

        all_patients_aggregated["g_min"]   = min(all_count[patient]["min"] for patient in all_count)
        all_patients_aggregated["g_max"]   = max(all_count[patient]["max"] for patient in all_count)
        sum_mean                 = sum(all_count[patient]["mean"] for patient in all_count)
        all_patients_aggregated["g_mean"]  = sum_mean / len(all_count)    

    # all_patients_aggregated = {'F': 802,
    #                             'N': 90502,
    #                             'S': 2777,
    #                             'V': 7226,
    #                             'Q': 8031,
    #                             'g_min': np.float64(-13.04264919165635),
    #                             'g_max': np.float64(13.320118149995809),
    #                             'g_mean': np.float64(0.01151252087416888)}

    # Step 3: aggregate values while iteratively leaving one patient out
    all_count_agg = {patient: {"F": 0, "N": 0, "S": 0, "V": 0, "Q": 0, "g_min": 0.0, "g_max": 0.0, "g_mean": 0.0} for patient in dict_patients}
    counter = 0

    for patient_leavout in dict_patients:

        # leave current patient out
        all_count_copy = copy.deepcopy(all_count)
        del all_count_copy[patient_leavout] 

        # sum counts for all other patients
        for patient in all_count_copy:
            for key in ["F", "N", "S", "V", "Q"]:
                all_count_agg[patient_leavout][key] += all_count_copy[patient][key]

        # all_count_agg[patient_leavout]["g_min"]   = min(all_count_copy[patient]["min"] for patient in all_count_copy)
        # all_count_agg[patient_leavout]["g_max"]   = max(all_count_copy[patient]["max"] for patient in all_count_copy)
        # sum_mean                                  = sum(all_count_copy[patient]["mean"] for patient in all_count_copy)
        # all_count_agg[patient_leavout]["g_mean"]  = sum_mean / len(all_count_copy)

    # Step 4: calculate the ratios
    diff_f = all_patients_aggregated["F"] - min(all_count_agg[patient]["F"] for patient in all_count_agg)
    ratio_f = diff_f / all_patients_aggregated["F"]

    diff_n = all_patients_aggregated["N"] - min(all_count_agg[patient]["N"] for patient in all_count_agg)
    ratio_n = diff_n / all_patients_aggregated["N"]

    diff_s = all_patients_aggregated["S"] - min(all_count_agg[patient]["S"] for patient in all_count_agg)
    ratio_s = diff_s / all_patients_aggregated["S"]

    diff_v = all_patients_aggregated["V"] - min(all_count_agg[patient]["V"] for patient in all_count_agg)
    ratio_v = diff_v / all_patients_aggregated["V"]

    diff_q = all_patients_aggregated["Q"] - min(all_count_agg[patient]["Q"] for patient in all_count_agg)
    ratio_q = diff_q / all_patients_aggregated["Q"]

    max_ratio = max(ratio_f, ratio_n, ratio_s, ratio_v, ratio_q)

    # diff_min  = all_patients_aggregated["g_min"] + abs(max(all_count_agg[patient]["g_min"] for patient in all_count_agg))
    # diff_max  = all_patients_aggregated["g_max"] - min(all_count_agg[patient]["g_max"] for patient in all_count_agg)
    # diff_mean_min = all_patients_aggregated["g_mean"] - min(all_count_agg[patient]["g_mean"] for patient in all_count_agg)
    # diff_mean_max = max(all_count_agg[patient]["g_mean"] for patient in all_count_agg) - all_patients_aggregated["g_mean"]

    return max_ratio



In [ ]:
global_sensitivity = get_global_sensitivity(dict_samples, values, labels)

In [ ]:
# print(ratio_f)
# print(ratio_n)
# print(ratio_s)
# print(ratio_v)
# print(ratio_q)

# print(diff_min)
# print(diff_max)
# print(diff_mean_max)
# print(diff_mean_min)

# printed:
# 0.46384039900249374
# 0.035269938785883186
# 0.49729924378826074
# 0.13728203708829229
# 0.2593699414767775
# 0.2786123120283415
# -0.5793735458718228
# 1.765213766018011
# 0.0009201804631791544
# 0.0017975683917464888

0.46384039900249374
0.035269938785883186
0.49729924378826074
0.13728203708829229
0.2593699414767775
0.2786123120283415
-0.5793735458718228
1.765213766018011
0.0009201804631791544
0.0017975683917464888


Die Schwankungen in den Werten selbst sind nicht ausschlaggebend für die Anwendung der Klassifizierungen, daher werden die relativen Veränderungen an den Klassen durch die Herausnahme einzelner Patienten betrachtet. Die Sensitivität unterscheidet sich recht stark zwischen den Klasse, da sie unterschiedlich start vertreten sind und es daher ein Patient mehr oder weniger starken Einfluss hat.

Gewählt wird die maximale Sensitivität der Klassen: F,N,S,V (da Q im modelltraining nicht weiter in Betracht gezogen wird)
Somit ist die Sensitivität 0,497.